# IPEX-LLM

> [IPEX-LLM](https://github.com/intel-analytics/ipex-llm) 是一个用于在英特尔 CPU 和 GPU（例如配备 iGPU 的本地 PC、Arc、Flex 和 Max 等独立 GPU）上以极低延迟运行 LLM 的 PyTorch 库。

- [IPEX-LLM 在英特尔 GPU 上的应用](#ipex-llm-on-intel-gpu)
- [IPEX-LLM 在英特尔 CPU 上的应用](#ipex-llm-on-intel-cpu)

## IPEX-LLM 在英特尔 GPU 上的应用

本示例将介绍如何使用 LangChain 与 `ipex-llm` 交互，在英特尔 GPU 上进行文本生成。

> **注意**
>
> 建议仅使用配备英特尔 Arc A 系列 GPU（英特尔 Arc A300 系列或 Pro A60 除外）的 Windows 用户直接运行 Jupyter notebook 来执行“IPEX-LLM 在英特尔 GPU 上”部分。对于其他情况（例如 Linux 用户、英特尔 iGPU 等），建议在终端中使用 Python 脚本运行代码以获得最佳体验。

### 安装先决条件
为了受益于 IPEX-LLM 在英特尔 GPU 上的应用，需要完成几步工具安装和环境准备的先决条件步骤。

如果您是 Windows 用户，请访问 [在 Windows 上使用英特尔 GPU 安装 IPEX-LLM 指南](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_windows_gpu.md)，并按照 [安装先决条件](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_windows_gpu.md#install-prerequisites) 更新 GPU 驱动程序（可选）并安装 Conda。

如果您是 Linux 用户，请访问 [在 Linux 上使用英特尔 GPU 安装 IPEX-LLM](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_linux_gpu.md)，并按照 [**安装先决条件**](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_linux_gpu.md#install-prerequisites) 安装 GPU 驱动程序、Intel® oneAPI Base Toolkit 2024.0 和 Conda。

### 设置

在先决条件安装完成后，您应该已经创建了一个已安装所有先决条件的 conda 环境。**在此 conda 环境中启动 jupyter 服务**：

In [ ]:
%pip install -qU langchain langchain-community

在 Intel GPU 上本地运行 LLM，请安装 IEPX-LLM。

In [ ]:
%pip install --pre --upgrade ipex-llm[xpu] --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/xpu/us/

> **注意**
>
> 您也可以使用 `https://pytorch-extension.intel.com/release-whl/stable/xpu/cn/` 作为 extra-indel-url。

### 运行时配置

为达到最佳性能，建议根据您的设备设置多个环境变量：

#### 适用于配备 Intel Core Ultra 集成 GPU 的 Windows 用户

In [ ]:
import os

os.environ["SYCL_CACHE_PERSISTENT"] = "1"
os.environ["BIGDL_LLM_XMX_DISABLED"] = "1"

#### 适用于配备 Intel Arc A 系列 GPU 的 Windows 用户

In [ ]:
import os

os.environ["SYCL_CACHE_PERSISTENT"] = "1"

> **注意**
>
> 首次在 Intel iGPU/Intel Arc A300 系列或 Pro A60 上运行每个模型时，可能需要几分钟时间进行编译。
>
> 对于其他 GPU 类型，请参考 [此处](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Overview/install_gpu.md#runtime-configuration)（面向 Windows 用户）和 [此处](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Overview/install_gpu.md#runtime-configuration-1)（面向 Linux 用户）。

### 基本用法

In [ ]:
import warnings

from langchain.chains import LLMChain
from langchain_community.llms import IpexLLM
from langchain_core.prompts import PromptTemplate

warnings.filterwarnings("ignore", category=UserWarning, message=".*padding_mask.*")

指定模型的提示模板。在此示例中，我们使用 [vicuna-1.5](https://huggingface.co/lmsys/vicuna-7b-v1.5) 模型。如果您使用的是其他模型，请相应选择合适的模板。

In [ ]:
template = "USER: {question}\nASSISTANT:"
prompt = PromptTemplate(template=template, input_variables=["question"])

使用 `IpexLLM.from_model_id` 在本地加载模型。它将直接以 Huggingface 格式加载模型，并自动转换为低比特格式以进行推理。在初始化 IpexLLM 时将 `model_kwargs` 中的 `device` 设置为 `"xpu"`，以便将 LLM 模型加载到 Intel GPU。

In [ ]:
llm = IpexLLM.from_model_id(
    model_id="lmsys/vicuna-7b-v1.5",
    model_kwargs={
        "temperature": 0,
        "max_length": 64,
        "trust_remote_code": True,
        "device": "xpu",
    },
)

在链中使用它

In [ ]:
llm_chain = prompt | llm

question = "What is AI?"
output = llm_chain.invoke(question)

### 保存/加载低比特模型

另外，您可以将低比特模型保存到磁盘一次，然后使用 `from_model_id_low_bit` 而不是 `from_model_id` 来重新加载它以供将来使用——甚至可以在不同的机器上使用。这在空间方面非常有效，因为低比特模型比原始模型占用的磁盘空间要少得多。而且，`from_model_id_low_bit` 在速度和内存使用方面也比 `from_model_id` 更有效，因为它跳过了模型转换步骤。您同样可以将 `model_kwargs` 中的 `device` 设置为 `"xpu"`，以便将 LLM 模型加载到 Intel GPU。

要保存低比特模型，请按以下方式使用 `save_low_bit`。

In [ ]:
saved_lowbit_model_path = "./vicuna-7b-1.5-low-bit"  # path to save low-bit model
llm.model.save_low_bit(saved_lowbit_model_path)
del llm

Load the model from saved lowbit model path as follows.
> 请注意，低比特模型的保存路径仅包含模型本身，不包含分词器。如果您希望将所有内容集中管理，则需要手动从原始模型的目录中下载或复制分词器文件到保存低比特模型的位置。

In [ ]:
llm_lowbit = IpexLLM.from_model_id_low_bit(
    model_id=saved_lowbit_model_path,
    tokenizer_id="lmsys/vicuna-7b-v1.5",
    # tokenizer_name=saved_lowbit_model_path,  # copy the tokenizers to saved path if you want to use it this way
    model_kwargs={
        "temperature": 0,
        "max_length": 64,
        "trust_remote_code": True,
        "device": "xpu",
    },
)

在 Chains 中使用已加载的模型：

In [ ]:
llm_chain = prompt | llm_lowbit


question = "What is AI?"
output = llm_chain.invoke(question)

## 在 Intel CPU 上使用 IPEX-LLM

本示例将介绍如何使用 LangChain 与 `ipex-llm` 结合，在 Intel CPU 上进行文本生成。

### 设置

In [ ]:
# Update Langchain

%pip install -qU langchain langchain-community

在 Intel CPU 上本地运行 LLM 的 IEPX-LLM 安装指南：

#### 对于 Windows 用户：

In [ ]:
%pip install --pre --upgrade ipex-llm[all]

#### 对于 Linux 用户：

In [ ]:
%pip install --pre --upgrade ipex-llm[all] --extra-index-url https://download.pytorch.org/whl/cpu

### 基本用法

In [ ]:
import warnings

from langchain.chains import LLMChain
from langchain_community.llms import IpexLLM
from langchain_core.prompts import PromptTemplate

warnings.filterwarnings("ignore", category=UserWarning, message=".*padding_mask.*")

指定模型的提示模板。在本例中，我们使用 [vicuna-1.5](https://huggingface.co/lmsys/vicuna-7b-v1.5) 模型。如果您使用的是其他模型，请相应选择合适的模板。

In [ ]:
template = "USER: {question}\nASSISTANT:"
prompt = PromptTemplate(template=template, input_variables=["question"])

使用 `IpexLLM.from_model_id` 在本地加载ipexLLM模型。它将直接以Huggingface格式加载模型，并自动转换为低比特格式以进行推理。

In [ ]:
llm = IpexLLM.from_model_id(
    model_id="lmsys/vicuna-7b-v1.5",
    model_kwargs={"temperature": 0, "max_length": 64, "trust_remote_code": True},
)

在链中使用：

In [ ]:
llm_chain = prompt | llm

question = "What is AI?"
output = llm_chain.invoke(question)

### 保存/加载低精度模型

另外，您可以将低精度模型保存到磁盘一次，然后使用 `from_model_id_low_bit` 而不是 `from_model_id` 来重新加载它以供以后使用——甚至可以在不同机器上使用。这在空间效率方面很出色，因为低精度模型比原始模型占用的磁盘空间少得多。而且，由于 `from_model_id_low_bit` 跳过了模型转换步骤，因此在速度和内存使用方面也比 `from_model_id` 更高效。

要保存低精度模型，请按以下方式使用 `save_low_bit`：

In [ ]:
saved_lowbit_model_path = "./vicuna-7b-1.5-low-bit"  # path to save low-bit model
llm.model.save_low_bit(saved_lowbit_model_path)
del llm

如下加载模型从保存的低比特模型路径。

> 请注意，低比特模型保存的路径仅包含模型本身，而不包含分词器。如果您希望所有内容都在一个地方，您需要手动下载或将分词器文件从原始模型的目录复制到保存低比特模型的位置。

In [ ]:
llm_lowbit = IpexLLM.from_model_id_low_bit(
    model_id=saved_lowbit_model_path,
    tokenizer_id="lmsys/vicuna-7b-v1.5",
    # tokenizer_name=saved_lowbit_model_path,  # copy the tokenizers to saved path if you want to use it this way
    model_kwargs={"temperature": 0, "max_length": 64, "trust_remote_code": True},
)

在链中使用已加载的模型：

In [ ]:
llm_chain = prompt | llm_lowbit


question = "What is AI?"
output = llm_chain.invoke(question)